### 📌 Tiêu chí cần đáp ứng

### 🔧Cài thư viện dlib từ file.whl

In [1]:
pip install dlib-19.22.99-cp39-cp39-win_amd64.whl

Processing c:\users\acer\baitaploncv\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


### 🔍 Khai báo thư viện

In [22]:
import dlib
import cv2
import os
import numpy as np
from imutils import paths
import time

## 1️⃣ Thu thập dữ liệu (n ảnh từ camera)

1.1. Khởi tạo detector HOG của Dlib

In [23]:
detector = dlib.get_frontal_face_detector()

1.2. Chỉnh sửa thư mục lưu ảnh 

In [63]:
person_name = "tuan anh"  # Đổi tên theo người(viết không dấu)
output_dir = f"data/{person_name}"
os.makedirs(output_dir, exist_ok=True)

print('Đường dẫn ảnh: ' + output_dir + '/')

Đường dẫn ảnh: data/tuan anh/


1.3. Điều chỉnh cách thức lấy ảnh

In [64]:
img_count = 0 
MAX_IMAGES = 200 # Giới hạn số lượng ảnh cần lấy

print(f'Số lượng ảnh cần thu thập: {MAX_IMAGES} ảnh')

Số lượng ảnh cần thu thập: 200 ảnh


1.4. Bắt đầu thu thập ảnh (mở camera)

In [65]:
print('⏳Bắt đầu thu thập ảnh...')
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for i, face in enumerate(faces):
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        # Cắt khuôn mặt
        face_img = frame[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (150, 150))
        img_path = f"{output_dir}/{person_name}_{img_count}.jpg"
        cv2.imwrite(img_path, face_img)
        img_count += 1

        # Vẽ hình chữ nhật quanh mặt
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Dang tu dong thu thap du lieu - Nhan ESC de dung luu/thoat", frame)

    if cv2.waitKey(1) == 27 or img_count >= MAX_IMAGES:  # Bấm ESC để thoát
        break

cap.release()
cv2.destroyAllWindows()

print('✅Thu thập ảnh hoàn tất')

⏳Bắt đầu thu thập ảnh...
✅Thu thập ảnh hoàn tất


## 2️⃣ Trích xuất đặc trưng HOG + 128D embedding & gán nhãn

2.1. Load models của Dlib

In [66]:
detector = dlib.get_frontal_face_detector()
# Dự đoán 68 điểm (shape_predictor)
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # cần tải trước
# Trích xuất vector 128D (resnet)
facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")  # cần tải trước

print('✅Hoàn tắt load models')

✅Hoàn tắt load models


2.2. Duyệt qua thư mục dataset

In [67]:
dataset_path = "data"
embeddings = []
labels = []
print('⏳Đang xử lý dữ liệu...')
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    for img_path in paths.list_images(person_path):
        image = cv2.imread(img_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        if len(faces) != 1:
            continue  # bỏ qua ảnh không rõ mặt

        shape = sp(gray, faces[0]) # Dự đoán 68 điểm (shape_predictor)
        face_descriptor = facerec.compute_face_descriptor(image, shape) # Trích xuất vector 128D (resnet)
        embeddings.append(np.array(face_descriptor))
        labels.append(person_name)

print('✅Hoàn tất duyệt dữ liệu')

⏳Đang xử lý dữ liệu...
✅Hoàn tất duyệt dữ liệu


2.3. Lưu dữ liệu huấn luyện

In [68]:
np.save("face_embeddings.npy", embeddings)
np.save("face_labels.npy", labels)

print("✅Đã lưu xong đặc trưng khuôn mặt và nhãn!")

✅Đã lưu xong đặc trưng khuôn mặt và nhãn!


## 3️⃣ Nhận diện khuôn mặt realtime

3.1. Load model

In [69]:
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

print("✅Hoàn tất load model")

✅Hoàn tất load model


3.2. Load dữ liệu đã huấn luyện

In [70]:
known_embeddings = np.load("face_embeddings.npy")
known_labels = np.load("face_labels.npy", allow_pickle=True)

print("✅Hoàn tất load dữ liệu train")

✅Hoàn tất load dữ liệu train


3.3. Hàm tính khoảng cách Euclidean

In [71]:
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)
# Ngưỡng khoảng cách: dưới ngưỡng là đúng người (tùy chỉnh nếu cần)
THRESHOLD = 0.6

print(f'Hàm tính khoảng cách ngưỡng: {THRESHOLD}')

Hàm tính khoảng cách ngưỡng: 0.6


3.4 Tùy chỉnh camera, log file

In [72]:
# Tạo màu khung
fixed_colors = [
    (0, 0, 255),     # Xanh dương
    (255, 0, 0),     # Đỏ
    (0, 255, 0),     # Xanh lá
    (255, 255, 0),   # Vàng
    (128, 0, 128),   # Tím
    (0, 0, 0),   # Đen
    (255, 255, 255),   # Trắng
]
color_map = {}
def get_fixed_color(name):
    if name not in color_map:
        idx = len(color_map) % len(fixed_colors)
        color_map[name] = fixed_colors[idx]
    return color_map[name]

In [73]:
# Chỉnh fps
target_fps = 15
frame_duration = 1.0 / target_fps
prev_time = 0

In [74]:
# File log để ghi lại người đã xuất hiện
log_file = "detected_people.log"
last_log_time = time.time()  # Lần ghi log cuối cùng (ban đầu là thời gian hiện tại)

def log_person(name):
    global last_log_time  # Dùng biến toàn cục để theo dõi thời gian cuối cùng ghi log
    current_time = time.time()

    # Kiểm tra nếu đã đủ 3 giây kể từ lần ghi log cuối
    if current_time - last_log_time >= 3:
        # Ghi log
        with open(log_file, "a", encoding="utf-8") as file:
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            file.write(f"{timestamp} - {name} đã xuất hiện\n")
        
        last_log_time = current_time  # Cập nhật thời gian ghi log lần cuối

In [75]:
# Overlay UI

# def draw_ui_overlay(frame, fps, names_in_frame, log_status):
#     height, width = frame.shape[:2]
#     cv2.rectangle(frame, (0, 0), (width, 100), (0, 0, 0), -1)
#     cv2.putText(frame, f"FPS: {fps:.2f}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#     cv2.putText(frame, f"People: {len(names_in_frame)}", (150, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
#     names_text = ", ".join(names_in_frame)
#     cv2.putText(frame, f"Names: {names_text}", (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
#     cv2.putText(frame, f"Log: {log_status}", (10, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 255, 255), 2)
#     if "Unknown" in names_in_frame:
#         cv2.putText(frame, "⚠️ WARNING: Unknown Face!", (width - 300, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


3.4. Bắt đầu nhận diện khuôn mặt (mở camera)

In [76]:
print("⏳Bắt đầu nhận diện khuôn mặt...")
cap = cv2.VideoCapture(0)

while True:
    now = time.time()
    elapsed = now - prev_time

    if elapsed < frame_duration:
        continue  # Chờ nếu chưa đến thời gian cho frame tiếp theo

    prev_time = now
    fps = 1.0 / elapsed
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

#     names_in_frame = []

    for face in faces:
        shape = sp(gray, face)
        face_descriptor = facerec.compute_face_descriptor(frame, shape)
        face_embedding = np.array(face_descriptor)

        # So khớp với dữ liệu đã biết
        distances = [euclidean_distance(face_embedding, emb) for emb in known_embeddings]
        min_dist = min(distances)
        min_index = distances.index(min_dist)

        name = "Unknown"
        if min_dist < THRESHOLD:
            name = known_labels[min_index]

        # Hiển thị kết quả
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        color = get_fixed_color(name)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{name} ({min_dist:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        # Ghi log người xuất hiện
        log_person(name)
#         names_in_frame.append(name)
        # Vẽ FPS lên ảnh
        fps_text = f"FPS: {fps:.2f}"
        cv2.putText(frame, fps_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
#         end_time = time.time()
#         fps = 1 / (end_time - start_time)
#         start_time = end_time
#         draw_ui_overlay(frame, fps, names_in_frame, "✓ Đã ghi log")

    cv2.imshow("Nhan dien khuon mat bang Dlib/HoG - Nhan ESC de thoat", frame)

    key = cv2.waitKey(1)

    if key == 27:
        break

    elif key == ord('n'):
        # Nhập tên
        person_name = input("Nhập tên người mới: ")

        # Tạo thư mục ảnh nếu cần
        os.makedirs(f'dataset/{person_name}', exist_ok=True)

        # Thu thập 20 ảnh
        for i in range(20):
            ret, frame = cap.read()
            faces = detector(frame)
            for face in faces:
                shape = sp(frame, face)
                face_img = frame[face.top():face.bottom(), face.left():face.right()]
                cv2.imwrite(f'dataset/{person_name}/{person_name}_{i}.jpg', face_img)
            time.sleep(0.1)

        # Trích xuất vector 128D cho từng ảnh
        for img_file in os.listdir(f'dataset/{person_name}'):
            img = cv2.imread(f'dataset/{person_name}/{img_file}')
            dets = detector(img)
            for det in dets:
                shape = sp(img, det)
                face_descriptor = facerec.compute_face_descriptor(img, shape)
                known_faces.append(np.array(face_descriptor))
                known_names.append(person_name)

        print(f"✅ Đã thêm {person_name} vào danh sách nhận diện.")
cap.release()
cv2.destroyAllWindows()
print("✅Hoàn tất nhận diện khuôn mặt!")

⏳Bắt đầu nhận diện khuôn mặt...
✅Hoàn tất nhận diện khuôn mặt!


Nhóm 4

Dương Ngọc Anh

Lê Phú Hào